<a href="https://colab.research.google.com/github/Jack0karev/konoha/blob/master/Contest_correct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.dropbox.com/s/2oz8zxrm05tsx3u/train.json.zip
!wget https://www.dropbox.com/s/07dgpnsit9a1749/items.json.zip
!wget https://www.dropbox.com/s/k16x0iwggvzp6j2/random_benchmark.csv.zip
!unzip train.json.zip
!unzip items.json.zip
!unzip random_benchmark.csv.zip

In [0]:
!ls

# Загрузка данных

In [0]:
import pandas as pd
import numpy as np
import json

random_benchmark = pd.read_csv('random_benchmark.csv')

from tqdm.auto import tqdm
items_list=[]
with tqdm(open('./items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list).set_index('itemId')

        

In [0]:
%%time
rows = []
columns = []
values = []
counter = 0
with open('train.json') as f:
    for line in f:
        counter += 1
        item = json.loads(line)
        for key in list(item['trainRatings'].keys()):
          rows.append(item['userId'])
          columns.append(int(key))
          values.append(item['trainRatings'][key])


# Item embeddings


In [0]:
item_ids = []
titles = [] 
with tqdm(open("items.json")) as f:
    for i, line in enumerate(f):
        item = json.loads(line)
        item_ids.append(item['itemId'])
        titles.append(item['title'])
        if item.get('image') is None:
            item['image'] = [0 for _ in range(96)]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=False, min_df = 10, max_df = 0.01)
tfidf_titles = vectorizer.fit_transform(titles)

# Топ контент (scoare: 0.10421)

In [0]:
ranked_dict = {}
for column, value in zip(columns, values):
   if value != 0: 
    if str(column) not in ranked_dict:
      ranked_dict[str(column)] = value
    else:
      ranked_dict[str(column)] += value


In [0]:
def function(x):
  if str(x.itemId) not in ranked_dict:
    return 0
  else:
    return ranked_dict[str(x.itemId)]
random_benchmark['ratings'] = random_benchmark.apply(function ,axis=1)  
random_benchmark.head()

In [0]:
random_benchmark.sort_values(['userId','ratings'], ascending=[True,False])


In [0]:
df = random_benchmark.sort_values(['userId','ratings'], ascending=[True,False])
total = df[['userId', 'itemId']]


In [0]:
total = total.reset_index()[['userId','itemId']]

In [0]:
from google.colab import files
total.to_csv('top_content.csv', index=False) 
files.download('top_content.csv')

# Коллаборативная фильтрация крашится :) (Даже код не сохранил, но он такой же как с семинара)
# Можно не запускать

In [0]:
ratings_m = ratings.values

In [0]:

similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
        
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        
        if np.sum(mask_uv) == 0:
            continue
            
        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]
        
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

In [0]:
prediction_user_based = []
for i in tqdm_notebook(range(len(similarity_users))):
    users_sim = similarity_users[i] > 0
    if len(users_sim) == 0:
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based

# Безуспешная попытка использовать KNN :) (Слишком долго делается один предикт)

In [0]:
items_list=[]
with tqdm(open('items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list)

In [0]:
items.head()

In [0]:
item_to_idx = {
    item: i for i, item in 
    enumerate(list(items.set_index('itemId').loc[items.itemId].title))}

In [0]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz


In [0]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(tfidf_titles)

In [0]:
def fuzzy_matching(mapper, fav_title, verbose=True):
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_title.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

In [0]:
def make_recommendation(model_knn, data, mapper, fav_title, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input title:', fav_title)
    idx = fuzzy_matching(mapper, fav_title, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_title))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [0]:
my_favorite = items.title[0]

make_recommendation(
    model_knn=model_knn,
    data=tfidf_titles,
    fav_title=my_favorite,
    mapper=item_to_idx,
    n_recommendations=10)



# LightFM (score: 0.12750)

In [0]:
from scipy.sparse import coo_matrix
row_item = []
column_item = []
value_item = []
for i in range(items.shape[0]):
  row_item.extend([i] * 96)
  column_item.extend(list(range(96)))
  for value in items.image[i]:
    value_item.append(value)
item_sparse = coo_matrix((value_item, (row_item, column_item)), shape=(items.shape[0], 96))

In [0]:
user_ids = np.array(rows)
item_ids = np.array(columns)
ratings = np.array(values)
from scipy.sparse import coo_matrix
ratings_sparse = coo_matrix((ratings, (user_ids, item_ids)), shape=(counter, items.shape[0]))

In [0]:
!pip install lightfm
from lightfm import LightFM

In [0]:
NUM_COMPONENTS = 30
NUM_EPOCHS = 20
ITEM_ALPHA = 1e-4
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
                no_components=NUM_COMPONENTS)
model.fit(ratings_sparse, item_features = tfidf_titles,epochs=30, num_threads=4)

In [0]:
nump_user = random_benchmark.userId.to_numpy()
nump_item = random_benchmark.itemId.to_numpy()

In [0]:
predictions = model.predict(nump_user, nump_item, tfidf_titles)

In [0]:
random_benchmark['ratings'] = predictions
df = random_benchmark.sort_values(['userId','ratings'], ascending=[True,False]).reset_index()
df.head()

In [0]:
total = df[['userId', 'itemId']]
from google.colab import files
total.to_csv('lightFM_5.csv', index=False) 
files.download('lightFM_5.csv')

# Matrix Factorization model (score: 0.09330)

In [0]:
def get_batch(user_ids, item_ids, ratings, feature_embs,  batch_size=32):
    ind = np.random.randint(0, len(user_ids), size=(batch_size,))
    embs = []
    for i in item_ids[ind]:
      embs.append(tfidf_titles.getrow(i).toarray())
    feature_embs = torch.tensor(np.array(embs).reshape(batch_size, 29762))
    yield torch.tensor(user_ids[ind]), torch.tensor(item_ids[ind]), torch.tensor(ratings[ind]), feature_embs

In [0]:
import torch 
import torch.nn as nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_users, n_items, embedding_size, fearture_size):
        super(Model, self).__init__()
        self.emb1 = nn.Embedding(n_users, embedding_size)
        self.emb2 = nn.Embedding(n_items, embedding_size)
        self.hidden = nn.Linear(2 * embedding_size + fearture_size, 1)
    
    def forward(self, user, item, features):
        user_emb = self.emb1(user)
        item_emb = self.emb2(item)
        product = self.hidden(torch.cat([user_emb, item_emb, features.float()], dim = 1))
        return product

In [0]:
model = Model(user_ids.max() + 1, item_ids.max() + 1, 100, 29762)

In [0]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
step = 0
while step < 10000:
    for u, i, y, emb in get_batch(user_ids, item_ids, ratings, tfidf_titles):
        output = model.forward(u, i, emb)
        loss_data = loss(output, y.float())
        loss_data.backward()
        optimizer.step()
        step += 1

In [0]:
model.eval()

In [0]:
random_benchmark.shape

In [0]:
users = torch.tensor(random_benchmark.userId)
items_1 = torch.tensor(random_benchmark.itemId)

In [0]:
pred = model.forward(users[0].reshape(1, ), items_1[0].reshape(1,), torch.tensor(tfidf_titles.getrow(np.array(random_benchmark.itemId)[0]).toarray()))


# Испугался вдпуг все крашнится)


In [0]:
pred = model.forward(users[0].reshape(1, ), items_1[0].reshape(1,), torch.tensor(tfidf_titles.getrow(np.array(random_benchmark.itemId)[0]).toarray()))
for i in range(1, random_benchmark.shape[0]):
  pred_1 = model.forward(users[i].reshape(1, ), items_1[i].reshape(1,),torch.tensor(tfidf_titles.getrow(np.array(random_benchmark.itemId)[i]).toarray()))
  pred = torch.cat([pred, pred_1], dim = 0)

# Все крашнулось)

In [0]:
random_benchmark['ratings'] = pred
df = random_benchmark.sort_values(['userId','ratings'], ascending=[True,False]).reset_index()
df.head()

In [0]:
total = df[['userId', 'itemId']]
from google.colab import files
total.to_csv('MaxFac_final.csv', index=False) 
files.download('MaxFac_final.csv')